# Desenvolvendo Dimensão Vendedores 

- scd 1

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [3]:
 # Importando dados 

import pandas as pd 
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils.acessobanco import acessobanco

In [4]:
# Carrega as variáveis de ambiente do arquivo .env

load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")


True

In [5]:
# Cria a conexão com o banco usando a função acessobanco()

engine = acessobanco()

In [6]:
# Criando a df puxando do banco

df = pd.read_sql_table("vendedores",engine, schema="stg")

In [7]:
# Verificando os dados

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nome_vendedor  4 non-null      object
 1   regiao         4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [8]:
# Gravando dataframe vendedores

df_vendedores = df

In [9]:
# visualizando os dados

df_vendedores.head()

,nome_vendedor,regiao
0,Anna Andreadi,West
1,Chuck Magee,East
2,Kelly Williams,Central
3,Cassandra Brandow,South


In [10]:
# Criando um id para o modo de envio
df_vendedores['id_vendedor'] = range(1,len(df_vendedores) + 1)

In [11]:
# Conferindo a implementação os id

df_vendedores

,nome_vendedor,regiao,id_vendedor
0,Anna Andreadi,West,1
1,Chuck Magee,East,2
2,Kelly Williams,Central,3
3,Cassandra Brandow,South,4


In [12]:
# Organizando ordens das tabelas 

df_venedores = df_vendedores[['id_vendedor','nome_vendedor','regiao']]

In [13]:
# visualizando os dados

df_vendedores.head()

,nome_vendedor,regiao,id_vendedor
0,Anna Andreadi,West,1
1,Chuck Magee,East,2
2,Kelly Williams,Central,3
3,Cassandra Brandow,South,4


In [14]:
# Salvando os dados em uma tabela temp para aplicar o scd1

with engine.begin() as conn:
    df_vendedores.to_sql(
        "tmp_dim_vendedores",  # nome da tabela temporária no schema dw
        conn,               # conexão com o banco de dados
        schema="dw",        # schema onde a tabela será criada
        if_exists="replace", # substitui a tabela se já existir (garante dados atualizados)
        index=False,         # não cria coluna de índice do pandas
        method="multi",      # insere várias linhas por comando INSERT (melhor performance)
        chunksize=5000       # envia os dados em blocos de 5000 linhas (controle de memória)
    )



In [15]:
# Para verificarmos quantas linhas subiram

count = pd.read_sql("SELECT COUNT(*) as total_registros FROM dw.tmp_dim_vendedores", engine).iloc[0,0]
print(f"Total de registros na tabela: {count}")

Total de registros na tabela: 4


In [16]:
# Insere apenas novo metodos

insert_sql = """
INSERT INTO dw.dim_vendedores_scd1 (id_vendedor, nome_vendedor,regiao)
SELECT t.id_vendedor, t.nome_vendedor, t.regiao
FROM dw.tmp_dim_vendedores AS t
LEFT JOIN dw.dim_vendedores_scd1 AS d ON d.id_vendedor = t.id_vendedor
WHERE d.id_vendedor IS NULL;
"""

with engine.begin() as conn:
    result = conn.execute(text(insert_sql))

print(f"Linhas inseridas: {result.rowcount}")

Linhas inseridas: 0
